## Portfolio Project 1: Regression model

**Task**<br>
Predict the sale price of a house 

**Problem**<br>
Real estate agents need help optimizing their day-to-day operations. Being able to predict the price of a property can help give their clients **insight** on what response to expect from the market.

**Solution**<br>
Use ML(linear regression) to predcit the price of a house 

**Data /description is avaliable at https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data**

In [1]:
%config IPCompleter.greedy=True
%load_ext watermark
%watermark -d -u -a 'Micaela' -v -p numpy,scipy,matplotlib,sklearn,mlxtend

Micaela 
last updated: 2020-08-20 

CPython 3.8.3
IPython 7.16.1

numpy 1.18.5
scipy 1.5.0
matplotlib 3.2.2
sklearn 0.23.1
mlxtend 0.17.3


In [2]:
import numpy as np  #Linear algebra
import pandas as pd # data manipulation 
import seaborn as sns # data visualization
import matplotlib.pyplot as plt # plotting graphs 
%matplotlib inline 

## 1. Import data 

In [3]:
train = pd.read_csv('train.csv')
train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [4]:
test = pd.read_csv('test.csv')
test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [5]:
c_train = train.copy()
c_test = test.copy()
# for splitting purposes after cleaning 
c_train['train'] = 1
c_test['train'] = 0

In [6]:
# edit train and test simultaneously 
df = pd.concat([c_train, c_test], axis=0, sort = False)

## **2. Exploratory Data Analysis (EDA)**

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2919 entries, 0 to 1458
Data columns (total 82 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             2919 non-null   int64  
 1   MSSubClass     2919 non-null   int64  
 2   MSZoning       2915 non-null   object 
 3   LotFrontage    2433 non-null   float64
 4   LotArea        2919 non-null   int64  
 5   Street         2919 non-null   object 
 6   Alley          198 non-null    object 
 7   LotShape       2919 non-null   object 
 8   LandContour    2919 non-null   object 
 9   Utilities      2917 non-null   object 
 10  LotConfig      2919 non-null   object 
 11  LandSlope      2919 non-null   object 
 12  Neighborhood   2919 non-null   object 
 13  Condition1     2919 non-null   object 
 14  Condition2     2919 non-null   object 
 15  BldgType       2919 non-null   object 
 16  HouseStyle     2919 non-null   object 
 17  OverallQual    2919 non-null   int64  
 18  OverallC

Some columns that have initially caught my attention due to missing values: **Alley, PoolQC, Fence, MiscFeature<br>**
These columns appear to have many missing values. However, we must further explore the data for missing values python may not recognize (none, n/a, etc.)

**SalePrice is our target variable (y)**

## **3. Cleaning**

**Dropping columns**

Let's take a look at the columns previously mentioned

In [8]:
high_NAN = df[['Alley','PoolQC','Fence', 'MiscFeature']]
high_NAN.isnull().sum()/2919*100

Alley          93.216855
PoolQC         99.657417
Fence          80.438506
MiscFeature    96.402878
dtype: float64

All of the high_NAN features have over 80% missing data<br>
This indicates we need to drop these columns 

In [9]:
df_copy = df.drop(['Alley','PoolQC','Fence', 'MiscFeature'], axis=1)

In [10]:
df_copy.shape

(2919, 78)

Let's seperate the columns into object and numerical sets

In [11]:
obj_cols = df_copy.select_dtypes(include = ['object'])
num_cols = df_copy.select_dtypes(exclude = ['object'])

**Cleaning object columns**

In [13]:
obj_cols.isnull().sum().sort_values(ascending = False)[0:20]

FireplaceQu     1420
GarageCond       159
GarageQual       159
GarageFinish     159
GarageType       157
BsmtCond          82
BsmtExposure      82
BsmtQual          81
BsmtFinType2      80
BsmtFinType1      79
MasVnrType        24
MSZoning           4
Utilities          2
Functional         2
Electrical         1
KitchenQual        1
SaleType           1
Exterior2nd        1
Exterior1st        1
LandContour        0
dtype: int64

It's critical that we understand why these values are missing<br>
is it because that feature was absent in the house? or was it not recorded for unknown reasons?

In [41]:
fill_none =['FireplaceQu','GarageCond','GarageQual','GarageFinish',
                      'GarageType','BsmtCond','BsmtExposure','BsmtQual','BsmtFinType2','BsmtFinType1','MasVnrType']
obj_cols[fill_none] = obj_cols[fill_none].fillna('None')

C:\Users\micae\Anaconda_info\lib\site-packages\pandas\core\frame.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


Fill the rest with their most frequent value

In [42]:
fill_mode = ['MSZoning','Utilities','Functional','Electrical','KitchenQual','SaleType','Exterior2nd','Exterior1st']
for column in fill_mode: 
    obj_cols[column] = obj_cols[column].fillna(obj_cols[column].mode()[0])

<ipython-input-42-ee1d6c4bd293>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  obj_cols[column] = obj_cols[column].fillna(obj_cols[column].mode()[0])


In [44]:
obj_cols.head()

,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,...,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,SaleType,SaleCondition
0,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,...,Gd,Typ,None,None,None,None,None,Y,WD,Normal
1,RL,Pave,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,...,TA,Typ,None,None,None,None,None,Y,WD,Normal
2,RL,Pave,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,...,Gd,Typ,None,None,None,None,None,Y,WD,Normal
3,RL,Pave,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,...,Gd,Typ,None,None,None,None,None,Y,WD,Abnorml
4,RL,Pave,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,...,Gd,Typ,None,None,None,None,None,Y,WD,Normal


**<p style="color:red"> Edit warnings later**

**Clean numeric columns**

In [49]:
num_cols.isnull().sum().sort_values(ascending = False)[1:13]

LotFrontage     486
GarageYrBlt     159
MasVnrArea       23
BsmtFullBath      2
BsmtHalfBath      2
GarageCars        1
TotalBsmtSF       1
BsmtUnfSF         1
BsmtFinSF2        1
GarageArea        1
BsmtFinSF1        1
LowQualFinSF      0
dtype: int64

fill LotFrontage with median <br>
fil GarageYrBlt with year the house was built<br>
fill rest with 0 because it's 
**Structurally missing data**<br>

In [57]:
num_cols['LotFrontage'] = num_cols['LotFrontage'].fillna(num_cols['LotFrontage'].median())

<ipython-input-57-08c53407f916>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  num_cols['LotFrontage'] = num_cols['LotFrontage'].fillna(num_cols['LotFrontage'].median())
